#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           48 non-null     int64  
 1   producao       48 non-null     float64
 2   produtividade  48 non-null     int64  
 3   area           47 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 1.6 KB


### Avaliando a produção de soja

In [4]:
df.drop(['produtividade','area'], axis = 1,inplace=True)
df.head(10)


,date,producao
0,1977,19256.0
1,1978,14017.0
2,1979,16514.0
3,1980,19435.0
4,1981,21284.0
5,1982,21605.0
6,1983,19015.0
7,1984,21178.0
8,1985,21175.0
9,1986,20265.0


In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

      producao
date          
1977   19256.0
1978   14017.0
1979   16514.0
1980   19435.0
1981   21284.0
1982   21605.0
1983   19015.0
1984   21178.0
1985   21175.0
1986   20265.0


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Checking Train-Test Split

In [9]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [10]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [11]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

### theta 

In [12]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 9},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(theta, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.3737,1.6817,14644.0852,14644.0852,0.1681,0.1551
1,2021.0000,2.9675,2.1230,18936.0601,18936.0601,0.1674,0.1827
2,2022.0000,3.6853,2.6075,25125.2204,25125.2204,0.1905,0.2106
Mean,nan,3.0088,2.1374,19568.4552,19568.4552,0.1753,0.1828
SD,nan,0.5362,0.3781,4302.2081,4302.2081,0.0107,0.0226


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,125519.2050
2025,126505.2741
2026,127491.3431
2027,128477.4122
2028,129463.4813
2029,130449.5504
2030,131435.6195
2031,132421.6885
2032,133407.7576


###  ADA

In [13]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,0.4619,0.3273,2849.6487,2849.6487,0.0327,0.0322
1,2021.0000,3.1850,2.2786,20323.7071,20323.7071,0.1796,0.1974
2,2022.0000,5.2892,3.7423,36060.5925,36060.5925,0.2734,0.3167
Mean,nan,2.9787,2.1161,19744.6494,19744.6494,0.1619,0.1821
SD,nan,1.9761,1.3989,13564.4923,13564.4923,0.0991,0.1167


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## exp_smooth

In [14]:
exp_smooth  = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:27:46
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.8110,1.9915,17341.4091,17341.4091,0.1991,0.1811
1,2021.0000,1.5896,1.1372,10143.2527,10143.2527,0.0897,0.0939
2,2022.0000,3.2489,2.2987,22150.0505,22150.0505,0.1679,0.1833
Mean,nan,2.5498,1.8091,16544.9041,16544.9041,0.1522,0.1528
SD,nan,0.7021,0.4914,4934.0054,4934.0054,0.0460,0.0417


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ets

In [15]:

ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.9277,2.0742,18061.5242,18061.5242,0.2074,0.1879
1,2021.0000,1.5866,1.1351,10124.3440,10124.3440,0.0895,0.0937
2,2022.0000,3.2062,2.2686,21859.4735,21859.4735,0.1657,0.1807
Mean,nan,2.5735,1.8259,16681.7806,16681.7806,0.1542,0.1541
SD,nan,0.7070,0.4949,4889.1775,4889.1775,0.0488,0.0428


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [16]:
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})              

## Avaliando a Produtividade

In [3]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [4]:
df.drop(['producao','area'], axis = 1,inplace=True)
df.head(10)

,date,produtividade
0,1977,1632
1,1978,1276
2,1979,1461
3,1980,1665
4,1981,1752
5,1982,1692
6,1983,1631
7,1984,1735
8,1985,1773
9,1986,1549


In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

      produtividade
date               
1977           1632
1978           1276
1979           1461
1980           1665
1981           1752
1982           1692
1983           1631
1984           1735
1985           1773
1986           1549


In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,produtividade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(48, 1)"
5,Transformed data shape,"(48, 1)"
6,Transformed train set shape,"(47, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [9]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [10]:
top4 = exp.compare_models(n_select = 4)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.3936,1.0270,476.2745,476.2745,0.1005,0.0940,0.5167
polytrend,Polynomial Trend Forecaster,1.4195,1.0467,489.0312,489.0312,0.1004,0.0958,4.8533
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.6875,1.2434,572.2115,572.2115,0.1238,0.1114,0.1833
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.7327,1.2755,584.5487,584.5487,0.1286,0.1157,0.1767
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.7363,1.2795,598.5746,598.5746,0.1226,0.1168,0.1667
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.7409,1.2825,588.7891,588.7891,0.1285,0.1144,0.2467
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7085,622.7085,0.1275,0.1215,0.1433
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7110,622.7110,0.1275,0.1215,0.1600
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7093,622.7093,0.1275,0.1215,0.8033
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.8058,1.3305,622.7110,622.7110,0.1275,0.1215,0.8200


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [25]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})


####  ada

In [40]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(ada_cds_dt, fh = 9)

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,5973.0459
2025,5981.6852
2026,5760.1371
2027,5672.9651
2028,5787.0896
2029,5886.0281
2030,5983.9519
2031,6081.8757
2032,6179.7995


In [ ]:
#### lightgbm_cds_dt

In [13]:
lightgbm_cds_dt = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(lightgbm_cds_dt, fh = 7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:20:02
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,2.8562,2.1048,951.3643,951.3643,0.2179,0.1965
1,2021.0000,0.3756,0.2753,132.2646,132.2646,0.0252,0.0256
2,2022.0000,0.9490,0.7008,345.1946,345.1946,0.0583,0.0600
Mean,nan,1.3936,1.0270,476.2745,476.2745,0.1005,0.0940
SD,nan,1.0603,0.7817,347.0038,347.0038,0.0841,0.0738


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,5579.9026
2025,5677.8264
2026,5775.7502
2027,5873.6740
2028,5971.5978
2029,6069.5216
2030,6167.4454


In [39]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(ada_cds_dt, fh = 9)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:39:20
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,4.0809,3.0074,1359.3283,1359.3283,0.3113,0.2694
1,2021.0000,1.1871,0.8701,417.9844,417.9844,0.0797,0.0830
2,2022.0000,0.4932,0.3641,179.3786,179.3786,0.0303,0.0298
Mean,nan,1.9204,1.4139,652.2304,652.2304,0.1404,0.1274
SD,nan,1.5538,1.1455,509.3942,509.3942,0.1225,0.1027


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,5973.0459
2025,5981.6852
2026,5760.1371
2027,5672.9651
2028,5787.0896
2029,5886.0281
2030,5983.9519
2031,6081.8757
2032,6179.7995


#### gbr_cds_dt   Gradient Boosting w/ Cond. Deseasonalize & Detrending

In [41]:
gbr_cds_dt = exp.create_model('gbr_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = gbr_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,4.0588,2.9911,1351.9656,1351.9656,0.3096,0.2681
1,2021.0000,1.0982,0.8050,386.6955,386.6955,0.0738,0.0766
2,2022.0000,0.0412,0.0304,14.9849,14.9849,0.0025,0.0025
Mean,nan,1.7327,1.2755,584.5487,584.5487,0.1286,0.1157
SD,nan,1.7005,1.2536,563.4647,563.4647,0.1312,0.1119


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### rf_cds_dt   Random Forest w/ Cond. Deseasonalize & Detrending

In [42]:
rf_cds_dt = exp.create_model('rf_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = rf_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## ada_cds_dt   AdaBoost w/ Cond. Deseasonalize & Detrendin


In [30]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,3.9143,2.8846,1303.8389,1303.8389,0.2986,0.2598
1,2021.0000,0.4139,0.3034,145.7254,145.7254,0.0278,0.0282
2,2022.0000,0.7342,0.5422,267.0702,267.0702,0.0451,0.0461
Mean,nan,1.6875,1.2434,572.2115,572.2115,0.1238,0.1114
SD,nan,1.5800,1.1646,519.7051,519.7051,0.1238,0.1052


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


In [31]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 9},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(ets, fh = 9)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:30:07
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2020.0000,4.8554,3.5781,1617.2853,1617.2853,0.3703,0.3125
1,2021.0000,0.0051,0.0037,1.7989,1.7989,0.0003,0.0003
2,2022.0000,1.4149,1.0447,514.6305,514.6305,0.0869,0.0908
Mean,nan,2.0918,1.5422,711.2383,711.2383,0.1525,0.1346
SD,nan,2.0371,1.5010,674.0129,674.0129,0.1580,0.1311


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2024,5705.4437
2025,5805.9525
2026,5906.4613
2027,6006.9701
2028,6107.4789
2029,6207.9876
2030,6308.4964
2031,6409.0052
2032,6509.5140


### ALL GRAPHS

In [43]:
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = rf_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 9},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
## Avaliando a Area

In [ ]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbmilho_br.xls', sheet_name='Planilha1')
print(df.head(10))



In [ ]:
df.drop(['producao','produtividade'], axis = 1,inplace=True)
df.set_index('date',inplace=True)
df=df.sort_index(axis=0)
print(df.head(10))

In [ ]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

In [ ]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
top4 = exp.compare_models(n_select = 4)


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.


In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})